# Execute fuzzy SIRDS model to data from other coutries

In [8]:
import pandas as pd
from scipy.optimize import differential_evolution
from datetime import timedelta, datetime

from sirds_model import get_bounds_and_arguments, sirds_objective_function

## Reading data

In [9]:
df = pd.read_csv('data/output/df_ts_epidemic_episodes_other_countries.csv', index_col=0)
df.date = pd.to_datetime(df.date)

In [10]:
# df = df[df.location.isin(['Spain', 'United Kingdom', 'United States'])]
df = df[df.location.isin(['United States'])]

In [11]:
dict_max_date_to_fit = {'United States': df.date.max(), 'United Kingdom': df.date.max(), 'Spain': df.date.max()}

In [12]:
dict_cumulative_days_in_first_outbreak_to_max_bound_I0 = {'United States': 0, 'United Kingdom': None, 'Spain': 0}

## Execution SIRDS model

In [13]:
df = df.sort_values(by=['location', 'date'])

In [14]:
try:
    df_initial_results = pd.read_csv('data/output/execution_other_countries_results.csv')
except:
    df_initial_results = pd.DataFrame({'location':[]})

df_results = df_initial_results.copy()

DAYS_TO_RECOVERY = 8

DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR = 5
NUMBER_ESTIMATON_PER_COUNTRY = 21

for estimation in range(NUMBER_ESTIMATON_PER_COUNTRY):        
    for country in df.location.unique():
        print('country: '+country)
        
        max_date_to_fit = dict_max_date_to_fit[country]
        df_country = df[(df.location == country) & (df.date <= max_date_to_fit)].reset_index(drop=True)
        
        # Dates about begin time series and first case        
        if country == 'United Kingdom':            
            date_first_case = df_country[(df_country.ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA == 0) & (df_country.NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA == df_country.NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA.max())].iloc[0].date
        else:
            date_first_case = df_country[(df_country.estimated_onset_by_deaths > 0)].iloc[0].date
        
        cumulative_days_in_first_outbreak_to_max_bound_I0 = dict_cumulative_days_in_first_outbreak_to_max_bound_I0[country]

        # Period of analysis
        period_in_days = (max_date_to_fit - date_first_case).days + 1
                           
        estimations_performed = len(df_results[(df_results.location == country)])
    
        if (estimation == estimations_performed):
            print('estimation: '+str(estimation))
            
            rt_column = 'NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA'
            # rt_column = 'reproduction_rate'
            
            bounds, args = get_bounds_and_arguments(df_country, 'date', 'rate_new_deaths_moving_average', rt_column, 'rate_new_cases_moving_average', 'ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA', DAYS_TO_RECOVERY, date_first_case, max_date_to_fit, df_country.population.iloc[0], period_in_days, cumulative_days_in_first_outbreak_to_max_bound_I0)
            
            print(bounds)
            
            # Record the start time
            start_time = datetime.now()
            print(start_time)

            result = differential_evolution(sirds_objective_function, bounds, args=args, popsize=DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR, maxiter=10000, workers=4, updating='deferred')
    
            # Record the end time
            end_time = datetime.now()
    
            # Calculate the duration (in seconds) for the optimization
            duration = (end_time - start_time).total_seconds()
            print(duration)
            
            list_breakpoints_in_slow_transition = args[4]
            quantity_outbreaks = args[5]
            quantity_outbreak_adjustments = args[6]
    
            # Create a dictionary to store results
            estimation_result = {
                'location': country,
                'estimation': estimation,  # To differentiate between multiple estimations
                'result_fun': result.fun,
                'result_nfev': result.nfev,
                'result_nit': result.nit,
                'result_success': result.success,
                'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format start time as a string
                'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format end time as a string
                'duration_seconds': duration,  # Duration in seconds
                'pop_size': DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR,
                'period_in_days': period_in_days,
                'days_to_recovery': DAYS_TO_RECOVERY,
                'cumulative_days_in_first_outbreak_to_max_bound_I0': cumulative_days_in_first_outbreak_to_max_bound_I0,
                'date_first_case': date_first_case,
                'list_breakpoints_in_slow_transition': list_breakpoints_in_slow_transition,
                'x_initial_infected_population': result.x[0],
                'x_days_between_infections_0': result.x[1]
            }
            
            quantity_epidemic_periods_with_slow_transition = len(list_breakpoints_in_slow_transition) + 1
            
            for p in range(quantity_epidemic_periods_with_slow_transition):
                estimation_result['x_case_fatality_probability_'+str(p)] = result.x[2 + p]
                estimation_result['x_loss_immunity_in_days_'+str(p)] = result.x[2 + quantity_epidemic_periods_with_slow_transition + p]                                                 
            quantity_breakpoints = (quantity_outbreaks - 1) + (quantity_outbreak_adjustments)
            begin_breakpoint_parameters = 2 + 2*quantity_epidemic_periods_with_slow_transition
            for b in range(quantity_breakpoints):
                estimation_result['x_days_between_infections_'+str(b+1)] = result.x[begin_breakpoint_parameters + b]
                estimation_result['x_breakpoint_'+str(b+1)] = result.x[begin_breakpoint_parameters + quantity_breakpoints + b]
                estimation_result['x_transition_days_between_epidemic_periods_'+str(b+1)] = result.x[begin_breakpoint_parameters + 2*quantity_breakpoints + b]
    
            print(estimation_result)
    
            # Append the estimation result to the list            
            df_results = pd.concat([df_results, pd.DataFrame.from_records([estimation_result])])
    
            df_results.to_csv('data/output/execution_other_countries_results.csv', index=False)           

country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
country: United States
estimation: 20
[(0.00029460448895044606, 0.006799903245859607), (2.5081001199991104, 3.7632273242802667), (0.0006170718246882004, 0.013300999999999999), (0.0011582020238238707, 0.013300999999999999), (0.0011657040434613056, 0.008169197443064925), (0.00021223387581227712, 0.004746592568401134), (9.900000000000001e-05, 0.006593022105756891), (89.999999, 365.000001), (89.999999, 365.000001), (89.999999, 365.000001), (89.999999, 365.000001), (89.999999, 365.000001), (2.5081001199991104, 9.347746697905936), (2.164474

capi_return is NULL
capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.
Call-back cb_f_in_lsoda__user__routines failed.
capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


KeyboardInterrupt: 

In [ ]:
bounds[0]

In [ ]:
df_country[['date','rate_new_deaths_moving_average', 'rate_new_cases_moving_average', 'estimated_onset_by_deaths', 'NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA']]

In [ ]:
min(df_country[(df_country.rate_new_cases_moving_average > 0)].iloc[0].date, df_country[(df_country.estimated_onset_by_deaths > 0)].iloc[0].date)

In [ ]:
df_country[(df_country.rate_new_cases_moving_average > 0)].iloc[0].date

In [ ]:
df_country[(df_country.estimated_onset_by_deaths > 0)].iloc[0].date

In [ ]:
outbreaks = df.groupby('ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA').agg({'date': ['min', 'max']})
outbreaks

In [ ]:
outbreaks.iloc[0][('date','min')]

In [ ]:
rt_in_outbreak = df_country[
    (df_country['date'] >= outbreaks.iloc[0][('date','min')]) &
    (df_country['date'] <= outbreaks.iloc[0][('date','max')])][rt_column].values
rt_in_outbreak

In [ ]:
df_results